In [1]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/ITM_Project/"

Mounted at /content/drive


# Setting

setup codes

In [2]:
!pip install pretty_midi pydub spleeter basic-pitch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
import os
from tqdm import tqdm
import pathlib
import glob
import pretty_midi

# cut mp3
from pydub import AudioSegment
# audio to midi
from basic_pitch.inference import predict, predict_and_save
from basic_pitch import ICASSP_2022_MODEL_PATH

path definition

In [4]:
deam_path = path + 'data/DEAM/'
audio_path = deam_path + 'audio/'
mp3_cut_path = deam_path + '15_CUT_MP3/'
spleeter_audio_path = deam_path + 'spleeter_WAV'
wav2midi_path = deam_path + 'WAV2MIDI/'
final_path = deam_path + 'midi_files/'

# Helper Functions

In [5]:
def extract_song_id(filepath):
    res = filepath.split('/')   # res[-1] == sog_id.mp3
    return res[-1].split('.')[0]    # extension 제거

In [6]:
def extract_only_song_id(path):
    res = extract_song_id(path)
    return res.split('_')[0]    # basic-pitch annotation 제거

# Step 1: Cut MP3 File
Cut the first 15 seconds of each song since the DEAM dataset sampled VA scores after 15 second.

In [7]:
mp3_dir = pathlib.Path(audio_path)
mp3_files = glob.glob(str(mp3_dir/'*.mp3*'))

In [ ]:
for file in tqdm(mp3_files):
    # Open an mp3 file
    song = AudioSegment.from_file(file, format="mp3")

    # pydub does things in milliseconds
    start_time = 15 * 1000

    # song clip of 10 seconds from starting
    cut_15_seconds = song[start_time:]

    # extract song_id for file save
    song_id = extract_song_id(file)

    # save file
    cut_15_seconds.export(mp3_cut_path + song_id + ".mp3",
                            format="mp3")

100%|██████████| 1802/1802 [46:58<00:00,  1.56s/it]


In [8]:
# cut MP3 file
mp3_dir = pathlib.Path(mp3_cut_path)
mp3_files = glob.glob(str(mp3_dir/'*.mp3*'))
print('Number of files:', len(mp3_files))

Number of files: 1802


# Step 2: Split Each Instrument for Each MP3 file
use ***spleeter*** python module -> extract bass, piano, vocals, drums, and others

**Step 1. Extract instruments wav file from each mp3 file**

In [12]:
already_exist = os.listdir(spleeter_audio_path)
exist_file = [int(f) for f in already_exist]
len(set(exist_file))

1801

In [ ]:
for file in tqdm(mp3_files):
    if extract_song_id(file) not in exist_file:
        !spleeter separate -o /content/drive/MyDrive/ITM_Project/data/DEAM/spleeter_MIDI -p spleeter:5stems /content/drive/MyDrive/ITM_Project/data/DEAM/15_CUT_MP3/{extract_song_id(file)}.mp3

# Step 3: WAV to MIDI
use **basic-pitch** library (sampling rate = 22,050 Hz)
* convert each *song_id/instrument.wav* into *song_id/instrument.mid*

In [13]:
spleeter_dir = pathlib.Path(spleeter_audio_path)
spleeter_dirs = glob.glob(str(spleeter_dir/'*'))

In [16]:
already_exist = os.listdir(wav2midi_path)
exist_songs = [f for f in already_exist]
notyet_songs = [extract_only_song_id(dir) for dir in spleeter_dirs if extract_song_id(dir) not in exist_songs]
notyet_paths = [dir for dir in spleeter_dirs if extract_only_song_id(dir) not in exist_songs]
print(len(exist_songs))
print(len(notyet_songs), len(notyet_paths))

1801
0 0


In [ ]:
for dir in tqdm(notyet_paths):
    song_id = dir.split('/')[-1]
    output_dir = wav2midi_path + song_id + '/'
    os.makedirs(output_dir, exist_ok=True)

    wav_dir = pathlib.Path(dir)
    wav_files = glob.glob(str(wav_dir/'*.wav*'))

    predict_and_save(
        wav_files,
        output_dir,
        save_midi=True,
        sonify_midi=False,
        save_model_outputs=False,
        save_notes=False,
        model_path=str(ICASSP_2022_MODEL_PATH)
    )

  0%|          | 0/788 [00:00<?, ?it/s]


Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/737/piano.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/737/piano_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/737/vocals.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/737/vocals_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/737/bass.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/737/bass_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/737/drums.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/737/drums_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/737/other.wav...


  0%|          | 1/788 [00:27<5:55:07, 27.07s/it]



  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/737/other_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/757/piano.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/757/piano_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/757/drums.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/757/drums_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/757/vocals.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/757/vocals_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/757/bass.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/757/bass_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/757/other.wav...


  0%|          | 2/788 [00:40<4:08:33, 18.97s/it]

...



  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1541/other_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1546/vocals.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1546/vocals_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1546/piano.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1546/piano_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1546/drums.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1546/drums_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1546/bass.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1546/bass_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1546/other.wav...


100%|█████████▉| 787/788 [3:09:46<00:14, 14.44s/it]



  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1546/other_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1560/vocals.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1560/vocals_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1560/piano.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1560/piano_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1560/drums.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1560/drums_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1560/bass.wav...


  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1560/bass_basic_pitch.mid

Predicting MIDI for /content/drive/MyDrive/ITM_Project/spleeter_MIDI/1560/other.wav...


100%|██████████| 788/788 [3:09:57<00:00, 14.46s/it]



  Creating midi...
  💅 Saved to /content/drive/MyDrive/ITM_Project/WAV2MIDI/1560/other_basic_pitch.mid


In [17]:
wav2mid_dir = pathlib.Path(wav2midi_path)
wav2mid_dirs = glob.glob(str(wav2mid_dir/'*'))
print(len(wav2mid_dirs))

1801


# Step 4: Merge MIDI files
merge each single-instrument midi file into one multi-instrument midi file
mapping split instruments to ***midi-emotion*** instrument set

In [ ]:
# instrument mapping to each corresponding mid files
mid_instr_mapper = {"drums": "DRUMS",  "piano": "PIANO",
                    "other": "GUITAR",  "bass": "BASS", "vocals": "STRINGS" }

instrument_to_program = {"DRUMS": (0, True), "PIANO": (0, False), "GUITAR": (24, False),
                             "BASS": (32, False), "STRINGS": (48, False)}

In [ ]:
def get_init_tracks():
    tracks = {}
    for key, val in instrument_to_program.items():
            track = pretty_midi.Instrument(program=val[0], is_drum=val[1], name=key.lower())
            track.notes = []
            tracks.update({key: track})
    return tracks

In [ ]:
def get_instrument(mid):
    # file name format is '{instrument}_basic_pitch'
    file_instr = mid.split('_')[0]
    return mid_instr_mapper[file_instr]

In [ ]:
os.makedirs(final_path, exist_ok=True)

for dir in tqdm(wav2midi_path):
    song_id = dir.split('/')[-1]

    single_dir = pathlib.Path(dir)
    single_files = glob.glob(str(single_dir/'*.mid*'))

    # fill the tracks using single instrument midi files
    tracks = get_init_tracks()
    none_track = 0
    for file in single_files:
        pm = pretty_midi.PrettyMIDI(file)
        if len(pm.instruments) > 0:
            notes = pm.instruments[0].notes
            instrument = get_instrument(file.split('/')[-1])
            for note in notes:
                tracks[instrument].notes.append(pretty_midi.Note(velocity=note.velocity, pitch=note.pitch, start=note.start, end=note.end))
        else:
            none_track += 1
    # none_track이 5면 정상적인 MIDI 파일이 아니란 뜻 (no instrument)
    if none_track < len(single_files):
        # create one merged multi-instrument midi file for each song_id
        new_pm = pretty_midi.PrettyMIDI()
        new_pm.instruments += tracks.values()
        new_pm.write(final_path + song_id + '.mid')
    else:
        print(f"Song id #{song_id} has no instrument")

100%|██████████| 1801/1801 [28:13<00:00,  1.06it/s]


In [18]:
final_dir = pathlib.Path(final_path)
final_files = glob.glob(str(final_dir/'*'))
print(len(final_files))

1801
